# DeepLabCut Toolbox - Docker
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*

This notebook demonstrates the necessary steps to use DeepLabCut on your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the Docker container to:
- train a network
- evaluate a network
- analyze a novel video

This assumes you already have a project folder with labeled data! 


## Let's look at info about the Docker Environment:


In [ ]:
!nvcc --version

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
#let's make sure we see a GPU:
#tf.test.gpu_device_name()
#or
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

## Start here for training DeepLabCut and analyzing new videos!

In [ ]:
#GUIs don't work on in Docker (or the cloud), so label your data locally on your computer! 
#This notebook is for you to train and run video analysis!
import os
os.environ["DLClight"]="True"

In [ ]:
# now we are ready to train!
import deeplabcut
deeplabcut.__version__

### change to your path:

In [ ]:
path_config_file = '/home/mackenzie/DEEPLABCUT/DeepLabCut2.0/examples/Reaching-Mackenzie-2018-08-30/config.yaml' #change to yours!

## Note, if you moved the project, or downloaded this and using the demo code, you will need to edit the project path in the config.yaml file! 

Head over to the project folder to open the yaml file in any text editor (such as gedit in Ubuntu)

(description): project_path:  Full path of the project (edit if you need to move the project to a cluster/server/another computer or a different directory on your computer)

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

In [ ]:
deeplabcut.create_training_dataset(path_config_file,Shuffles=[1])

### now go edit the pose_cfg.yaml to make display_iters: low (i.e. 10), and save_iters: 500 (for demo's)

Now it is the time to start training the network!

## Start training
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#reset in case you started a session before...
#tf.reset_default_graph()

deeplabcut.train_network(path_config_file, shuffle=1, saveiters=1000, displayiters=10)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.3M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....


Firstly, if the above cell ran, you can stop it with "stop" or cntrl-C; you will get a Keyboard Interrupt error (this is fine!)

### A couple tips for possible troubleshooting (1): 

if you get **permission errors** when you run this step (above), first check if the weights downloaded. As some docker containers might not have privileges for this (it can be user specific). They should be under 'init_weights' (see path in the pose_cfg.yaml file). You can enter the DOCKER in the terminal:

see more here: https://github.com/MMathisLab/Docker4DeepLabCut2.0#using-the-docker-for-training-and-video-analysis

You can "cd" in the terminal to this location! i.e. copy and paste this in: **"cd usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/
"** 

And if you type "ls" to see the list of files, you should see the resnet:
**resnet_v1_50.ckpt**

If it is not there, run **"sudo download.sh"**
then change the permissions: **"sudo chown yourusername:yourusername resnet_v1_50.ckpt"**


### Troubleshooting (2): 
if it appears the training does not start (i.e. "Starting training..." does not print immediately),
then you have another session running on your GPU. Go check "nvidia-smi" and look at the process names. You can only have 1 per GPU!)

## Start evaluating
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
deeplabcut.evaluate_network(path_config_file)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

## There is an optional refinement step
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of DOCKER!** We recommend coming back to train and analyze videos... 
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
videofile_path = ['/home/mackenzie/DEEPLABCUT/DeepLabCut2.0/examples/Reaching-Mackenzie-2018-08-30/videos/MovieS2_Perturbation_noLaser_compressed.avi'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

## Create labeled video
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
#for making interactive plots.
#deeplabcut.plot_trajectories(path_config_file,videofile_path, plotting=True)

deeplabcut.plot_trajectories(path_config_file,videofile_path,showfigures=True)